In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df_date=pd.read_csv('dim_date.csv')

In [ ]:
df_date.info()
df_date.head()

In [ ]:
df_date['fiscal_year'].value_counts()

In [ ]:
df_district=pd.read_csv('dim_districts.csv')

In [ ]:
df_district.info()

In [ ]:
df_fact_stamps=pd.read_csv('fact_stamps.csv')

In [ ]:
df_fact_stamps.info()

In [ ]:
sns.histplot(x=df_fact_stamps['documents_registered_cnt'],kde=True)
print('The graph looks like a log-normal distribution')
#sns.boxplot(x=df_fact_stamps['documents_registered_cnt'])



In [ ]:
sns.histplot(x=df_fact_stamps['estamps_challans_cnt'])
#sns.boxplot(x=df_fact_stamps['estamps_challans_cnt'])

STAMP REGISTRATION

Q1 TOP AND BOTTOM FIVE DISTRICTS BASED ON REVENUE COLLECTED FROM DOCUMENT REGISTRATION AND E-CHALLANS

In [ ]:
print('TOP 5 DISTRICTS BASED ON REVENUE COLLECTED FROM DOCUMENTS REGISTERED')
df_fact_stamps_district=pd.merge(left=df_district,right=df_fact_stamps,on='dist_code',how='inner')
df_fact_stamps_district.groupby(by='district')['documents_registered_rev'].sum().sort_values(ascending=False).head(5)

In [ ]:
print('BOTTOM 5 DISTRICTS BASED ON REVENUE COLLECTED FROM DOCUMENTS REGISTERED')

df_fact_stamps_district.groupby(by='district')['documents_registered_rev'].sum().sort_values().head(5)

In [ ]:
print('TOP 5 DISTRICTS BASED ON TOTAL DOCUMENTED COLLECTED')

df_fact_stamps_district.groupby(by='district')['documents_registered_cnt'].sum().sort_values(ascending=False).head(5)

In [ ]:
print('BOTTOM 5 DISTRICTS BASED ON TOTAL DOCUMENTED COLLECTED')

df_fact_stamps_district.groupby(by='district')['documents_registered_cnt'].sum().sort_values().head(5)

In [ ]:
print('BOTTOM 5 DISTRICTS BASED ON REVENUE COLLECTED FROM ESTAMPS')

df_fact_stamps_district.groupby(by='district')['estamps_challans_rev'].sum().sort_values().head(5)

In [ ]:
print('TOP 5 DISTRICTS BASED ON REVENUE COLLECTED FROM ESTAMPS')


df_fact_stamps_district.groupby(by='district')['estamps_challans_rev'].sum().sort_values(ascending=False).head(5)

In [ ]:
#month
df_fact_stamps_district.groupby(by='month')['estamps_challans_rev'].sum().sort_values(ascending=False).head(5)

Q2. top 5 districts where e-stamps revenue contributes 
significantly more to the revenue than the documents in 2022

In [ ]:
from datetime import datetime
df_fact_stamps_district['date']=df_fact_stamps_district['month'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d'))
df_fact_stamps_district['year']=df_fact_stamps_district['date'].apply(lambda x:x.year)
"""""
left=df_fact_stamps_district.groupby(by='district','')['documents_registered_rev'].sum().to_frame().reset_index()
right=df_fact_stamps_district.groupby(by='district')['estamps_challans_rev'].sum().to_frame().reset_index()

pd.merge(left=left,right=right,on='district',how='inner')[pd.merge(left=left,right=right,on='district',how='inner')['estamps_challans_rev']>pd.merge(left=left,right=right,on='district',how='inner')['documents_registered_rev']]
"""""

left=df_fact_stamps_district.groupby(by=['district','year'])['documents_registered_rev'].sum().to_frame().reset_index()
right=df_fact_stamps_district.groupby(by=['district','year'])['estamps_challans_rev'].sum().to_frame().reset_index()

left[left['year']==2022]
print('TOP 5 DISTRICTS WHERE e-stamps revenue contributes significantly more to the revenue than the documents in FY 2022')
df_documents_estamps=pd.merge(left=left[left['year']==2022],right=right[right['year']==2022],on=['district','year'],how='inner')
df_documents_estamps['diff_estamp_registered']=df_documents_estamps['estamps_challans_rev']-df_documents_estamps['documents_registered_rev']
df_documents_estamps.sort_values(by='diff_estamp_registered',ascending=False).head(5)


In [ ]:
print('It shows that the relationship between revenue generated from document registration compare to the revenue generated from e-stamp challans across districts is linear')
sns.lmplot(data=df_fact_stamps_district,x='estamps_challans_rev',y='documents_registered_rev',hue='district')


In [ ]:
print('High co-relationship exists between estamps-revenue and revenue generated from documents')
print(sns.heatmap(np.corrcoef(df_fact_stamps_district['estamps_challans_rev'],df_fact_stamps_district['documents_registered_rev']),annot=True))

Q3. Alteration of e-Stamp challan count and document 
registration count pattern since the implementation of e-Stamp 
challan

In [ ]:

df_fact_stamps_district.groupby(by='date')['estamps_challans_rev'].sum().sort_values(ascending=False).plot()

print('from the graph we can stay that the revenue from estamps challans started from jan 2021 and was at peak in jan 2022 and is still increasing')

In [ ]:
df_fact_stamps_district.groupby(by='date')['documents_registered_rev'].sum().sort_values(ascending=False).plot()

print('from the graph we can say that revenue generated from documents registeration had an average value upto 2020 after which it started decreasing in july 2020 and after july 2021 revenue stated increasing')

In [ ]:
df_fact_stamps_district.groupby(by='date')['documents_registered_cnt'].sum().plot()

print('from the graph we can say that more people register upto 2020 but their revenue is smaller while after july 2021 the revenue generated is more with respect to total documents registered')
print('This indicates that fees of documents registeration was comparitevely less than todays time')


#df_fact_stamps_district.groupby(by='date')['documents_registered_cnt'].sum().index


In [ ]:
import seaborn as sns

# Sample data creation
data = {'Date':df_fact_stamps_district.groupby(by='date')['documents_registered_cnt'].sum().index,
        'documents_registered_cnt': df_fact_stamps_district.groupby(by='date')['documents_registered_cnt'].sum().values,
        'estamps_challans_cnt': df_fact_stamps_district.groupby(by='date')['estamps_challans_cnt'].sum().values}

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' column to datetime format

# Melt the DataFrame to make it suitable for Seaborn's lineplot
df_melted = df.melt(id_vars='Date', var_name='Feature', value_name='Value')

# Plotting using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(x='Date', y='Value', hue='Feature', data=df_melted, marker='o')

plt.title('Comparison of documents_registered_cnt and estamps_challans_cnt over Time')
plt.xlabel('Date')
plt.ylabel('Feature Values')
plt.legend()
plt.grid(True)
plt.show()


Insight--> The graph shows after implementation of estamps challans the count of e-stamp challan increased than count of document registered count


Suggestions--> This indicates people are more comfortable through online proces of estamps as it done anywhere at anytime. Government should take steps:

1. Online Platforms and Portals:
   - Establish user-friendly online platforms and portals for e-stamping services. This makes it easier for individuals and      businesses to access and use e-stamping services, reducing the need for physical visits to government offices.

2. Public Awareness:
   - Conduct public awareness campaigns to educate citizens, businesses, and professionals about the benefits of e-stamping. 

3. Accessibility and Inclusivity:
   - Ensure that e-stamping services are accessible to a wide range of users, including those in rural areas or with limited access to technology. This may involve establishing physical service centers or mobile solutions to reach a broader audience.

4. Data Security and Privacy:
   - Implement robust data security measures and privacy protections to safeguard sensitive information collected during the e-stamping process. Compliance with data protection regulations is crucial for user trust and legal compliance.

5. Electronic Payment Systems:
   - Facilitate electronic payment options for stamp duty fees. This reduces the need for physical cash transactions and enhances the efficiency of the payment process.

Q4. Categorize districts into three segments based on their stamp 
registration revenue generation during the fiscal year 2021 to 2022

In [ ]:
estamps_2021_2022=df_fact_stamps_district.groupby(by=['district','year'])['estamps_challans_rev'].sum().to_frame().reset_index()
estamps_2021_2022=estamps_2021_2022[(estamps_2021_2022['year']==2021) | (estamps_2021_2022['year']==2022)]
estamps_2021_2022=estamps_2021_2022.groupby(by='district')['estamps_challans_rev'].sum().to_frame().reset_index()

#estamps_2021_2022['estamps_challans_rev'].describe()

def cat_segments(x):
    if x>=1.247532e+08 and x<6.892411e+08:
        return 'low'
    elif x>=6.892411e+08 and x<2.616787e+09:
        return 'medium'
    elif x>=2.616787e+09:
        return 'high'
    
estamps_2021_2022['cat_segments']=estamps_2021_2022['estamps_challans_rev'].apply(lambda x:cat_segments(x))
estamps_2021_2022['estamps_challans_rev'].describe()

In [ ]:
estamps_2021_2022.head()
estamps_2021_2022.to_csv('estamps_2021_2022.csv')

In [ ]:
pd.crosstab(estamps_2021_2022['district'], estamps_2021_2022['cat_segments']) 
 

In [ ]:
print('Districts falling in low segments')
estamps_2021_2022[estamps_2021_2022['cat_segments']=='low']['district']

In [ ]:
print('Districts falling in medium segments')
estamps_2021_2022[estamps_2021_2022['cat_segments']=='medium']['district']

In [ ]:
print('Districts falling in high segments')
estamps_2021_2022[estamps_2021_2022['cat_segments']=='high']['district']

TRANSPORTATION

In [ ]:
#fact_transport.csv
df_fact_transport=pd.read_csv('fact_transport.csv')

In [ ]:
df_fact_transport.info()

In [ ]:
df_fact_transport.head()

In [ ]:
sns.histplot(x=df_fact_transport['fuel_type_petrol'])

In [ ]:
sns.histplot(x=df_fact_transport['fuel_type_diesel'])

In [ ]:
sns.histplot(x=df_fact_transport['vehicleClass_MotorCycle'])

THE GRAPH BELOW SHOWS HOW EACH FEATURE IS RELATED WITH OTHER

In [ ]:
sns.pairplot(df_fact_transport)

In [ ]:
df_fact_transport

Q5:PART_1 correlation between vehicle sales and 
specific months or seasons in different districts

In [ ]:
df_fact_transport['date']=df_fact_transport['month'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
df_fact_transport['month']=df_fact_transport['date'].apply(lambda x:x.month)

In [ ]:
df_fact_transport_district=pd.merge(left=df_fact_transport,right=df_district,on='dist_code',how='inner')

In [ ]:
#fuel_type_petrol	fuel_type_diesel	fuel_type_electric	fuel_type_others
df_fact_transport['total_sales']=df_fact_transport['fuel_type_petrol']+df_fact_transport['fuel_type_diesel']+df_fact_transport['fuel_type_electric']+df_fact_transport['fuel_type_others']

sns.barplot(x=df_fact_transport['month'],y=df_fact_transport['total_sales'])


FROM THE GRAPH ABOVE OCTOBER AD APRIL HAVE HIGHER AND LOWER SALES REPECTIVELY WITH RESPECT TO PETROL,DIESEL,ELECTRIC ETC 

In [ ]:
#df_fact_transport_district.groupby(by=['district','month'])['total_sales'].sum().to_frame().reset_index().sort_values(by='total_sales',ascending=False)

df_fact_transport_district.head()

Months that consistently show higher or lower sales rate 

FOR PETROL HIGH MONTH SALES ARE IN  OCTOBER AND LOW MONTH SALES ARE IN APRIL


In [ ]:
df_fact_transport_district.groupby(by='month')['fuel_type_petrol'].sum().to_frame().plot()

FOR DIESEL HIGH MONTH SALES ARE IN OCTOBER AND LOW MONTH SALES ARE IN APRIL

In [ ]:
df_fact_transport_district.groupby(by='month')['fuel_type_diesel'].sum().to_frame().plot()

FOR ELECTRIC HIGH MONTH SALES ARE IN MARCH AND LOW MONTH SALES ARE IN MAY

In [ ]:
df_fact_transport_district.groupby(by='month')['fuel_type_electric'].sum().to_frame().plot()

In [ ]:
df_fact_transport_district.groupby(by='date')['fuel_type_electric'].sum().to_frame().plot()

The sales of electric vehicles increased from 2019 to 2023 and is still increasing

In [ ]:
df_fact_transport_district.groupby(by='month')['fuel_type_others'].sum().to_frame().plot()

In [ ]:
import seaborn as sns

# Sample data creation
data = {'Date':df_fact_transport_district.groupby(by='date')['fuel_type_electric'].sum().index,
        'electric': df_fact_transport_district.groupby(by='date')['fuel_type_electric'].sum().values,
        'diesel': df_fact_transport_district.groupby(by='date')['fuel_type_diesel'].sum().values,
        'petrol':df_fact_transport_district.groupby(by='date')['fuel_type_petrol'].sum().values
       }

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' column to datetime format

# Melt the DataFrame to make it suitable for Seaborn's lineplot
df_melted = df.melt(id_vars='Date', var_name='Feature', value_name='Value')

# Plotting using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(x='Date', y='Value', hue='Feature', data=df_melted, marker='o')

plt.title('Comparison of various fuel type over Time')
plt.xlabel('Date')
plt.ylabel('Feature Values')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
print(sns.heatmap(np.corrcoef(df_fact_transport['month'],df_fact_transport['total_sales']),annot=True))
print('As the value of r is near to zero there is correlation')

Q6. Part1--> Distribution of vehicles vary by vehicle class 
(MotorCycle, MotorCar, AutoRickshaw, Agriculture) across different 
districts

In [ ]:
'''''
plt.figure(figsize=(20, 6))
plt.bar(df_fact_transport_district.groupby(by='district')['vehicleClass_MotorCycle'].sum().index, df_fact_transport_district.groupby(by='district')['vehicleClass_MotorCycle'].sum().values, color='r')
plt.bar(df_fact_transport_district.groupby(by='district')['vehicleClass_MotorCycle'].sum().index, df_fact_transport_district.groupby(by='district')['vehicleClass_MotorCar'].sum().values, color='b')
plt.bar(df_fact_transport_district.groupby(by='district')['vehicleClass_MotorCycle'].sum().index, df_fact_transport_district.groupby(by='district')['vehicleClass_AutoRickshaw'].sum().values, color='y')
plt.bar(df_fact_transport_district.groupby(by='district')['vehicleClass_MotorCycle'].sum().index, df_fact_transport_district.groupby(by='district')['vehicleClass_Agriculture'].sum().values, color='g')

plt.show()

df_vehicletype_district.plot(x='district', kind='bar', stacked=True,
        title='Stacked Bar Graph by dataframe')
plt.show()
'''''

df_fact_transport_district['year']=df_fact_transport_district['date'].apply(lambda x:x.year)
df_fact_transport_district_2022=df_fact_transport_district[df_fact_transport_district['year']==2022]
df_vehicletype_district=df_fact_transport_district_2022[['district','vehicleClass_MotorCycle','vehicleClass_MotorCar','vehicleClass_Agriculture','vehicleClass_AutoRickshaw']]

df_vehicletype_district=df_vehicletype_district.groupby(by='district').sum().reset_index()
plt.figure(figsize=(20, 10))
df_vehicletype_district.plot(x='district', kind='bar', stacked=True,
        title='distribution of vehicles vary  vehicle class ')
plt.show()

Q6 PART2--> districts with a predominant preference for a specific vehicle class

In [ ]:
"""""
fig = plt.figure(figsize =(10, 7))
plt.pie(df_vehicletype_district['vehicleClass_MotorCycle'], labels = df_vehicletype_district['district'])

"""""

import plotly.express as px
fig = px.pie(df_vehicletype_district, values='vehicleClass_MotorCycle', names='district')
fig.show()
print('Hyderabad, Medchal_Malkajgiri and Rangareddy are top 3 districts based motorcycle vehicle class')

In [ ]:
fig = px.pie(df_vehicletype_district, values='vehicleClass_MotorCar', names='district')
fig.show()
print('Rangareddy,Mechal_Malkajgiri and Hyderabad are top 3 districts based to MotorCar vehicle class')

In [ ]:
fig = px.pie(df_vehicletype_district, values='vehicleClass_Agriculture', names='district')
fig.show()
print('Nalgonda,siddhipet anf suryapet are top3 districts based on agriculture vehicle class')

In [ ]:
fig = px.pie(df_vehicletype_district, values='vehicleClass_AutoRickshaw', names='district')
fig.show()
print('Hyderabad,Sangareddy and Khammam are top 3 districts based on autorickshaw vehicle class')

In [ ]:
df_fact_transport_district_2022=df_fact_transport_district[(df_fact_transport_district['year']==2022)]
df_fact_transport_district_2021=df_fact_transport_district[(df_fact_transport_district['year']==2021)]
df_fact_transport_district_2022_fueltype=df_fact_transport_district_2022[['district','fuel_type_petrol','fuel_type_diesel','fuel_type_electric']].groupby(by='district').sum().reset_index()
df_fact_transport_district_2021_fueltype=df_fact_transport_district_2021[['district','fuel_type_petrol','fuel_type_diesel','fuel_type_electric']].groupby(by='district').sum().reset_index()

df_fact_transport_district_2021_2022=pd.merge(left=df_fact_transport_district_2021_fueltype,right=df_fact_transport_district_2022_fueltype,on='district',how='inner')
df_fact_transport_district_2021_2022=df_fact_transport_district_2021_2022.rename(columns={'fuel_type_petrol_x':'fuel_type_petrol_2021','fuel_type_diesel_x':'fuel_type_diesel_2021','fuel_type_electric_x':'fuel_type_electric_2021',
                                                                                        'fuel_type_petrol_y':'fuel_type_petrol_2022','fuel_type_diesel_y':'fuel_type_diesel_2022','fuel_type_electric_y':'fuel_type_electric_2022', })
df_fact_transport_district_2021_2022['petrol_diff']=df_fact_transport_district_2021_2022['fuel_type_petrol_2022']-df_fact_transport_district_2021_2022['fuel_type_petrol_2021']
df_fact_transport_district_2021_2022['diesel_diff']=df_fact_transport_district_2021_2022['fuel_type_diesel_2022']-df_fact_transport_district_2021_2022['fuel_type_diesel_2021']
df_fact_transport_district_2021_2022['electric_diff']=df_fact_transport_district_2021_2022['fuel_type_electric_2022']-df_fact_transport_district_2021_2022['fuel_type_electric_2021']
df_fact_transport_district_2021_2022['petrol_status']=df_fact_transport_district_2021_2022['petrol_diff'].apply(lambda x:'sales increased' if x>0 else 'decreased')
df_fact_transport_district_2021_2022['diesel_status']=df_fact_transport_district_2021_2022['diesel_diff'].apply(lambda x:'sales increased' if x>0 else 'decreased')
df_fact_transport_district_2021_2022['electric_status']=df_fact_transport_district_2021_2022['electric_diff'].apply(lambda x:'sales increased' if x>0 else 'decreased')



df_fact_transport_district_2021_2022

Q7. List down the top 3 and bottom 3 districts that have shown the highest 
and lowest vehicle sales growth during FY 2022 compared to FY 
2021

In [ ]:
df_fact_transport_district_2021_2022.sort_values(by='petrol_diff',ascending=False)[['district','petrol_status']].head(3)

In [ ]:
df_fact_transport_district_2021_2022.sort_values(by='petrol_diff',ascending=True)[['district','petrol_status']].head(3)

In [ ]:
df_fact_transport_district_2021_2022.sort_values(by='diesel_diff',ascending=False)[['district','diesel_status']].head(3)

In [ ]:
df_fact_transport_district_2021_2022.sort_values(by='diesel_diff',ascending=True)[['district','diesel_status']].head(3)

In [ ]:
df_fact_transport_district_2021_2022.sort_values(by='electric_diff',ascending=False)[['district','electric_status']].head(3)

In [ ]:
df_fact_transport_district_2021_2022.sort_values(by='electric_diff',ascending=True)[['district','electric_status']].head(3)

In [ ]:
df_fact_transport_district.info()

In [ ]:
#fact_TS_iPASS.csv
df_fact_ts_pass=pd.read_csv('fact_TS_iPASS.csv')

In [ ]:
df_fact_ts_pass.head()

In [ ]:
df_fact_ts_pass['date']=df_fact_ts_pass['month'].apply(lambda x:datetime.strptime(x,'%d-%m-%Y'))
df_fact_ts_pass['year']=df_fact_ts_pass['date'].apply(lambda x:x.year)
df_fact_ts_pass['month_ts_pass']=df_fact_ts_pass['date'].apply(lambda x:x.month)
df_fact_ts_pass_2022=df_fact_ts_pass[df_fact_ts_pass['year']==2022]
df_fact_ts_pass.info()

Q8.List down the top 5 sectors that have witnessed the most significant 
investments in FY 2022

In [ ]:
fig = plt.figure(figsize =(60, 7))
fig = px.pie(df_fact_ts_pass_2022, values='investment in cr', names='sector')
fig.show()
print('Real estate,plastic and rubber,pharmaceutical and chemical,solar and other beverages are top5 sectors based on investment')

In [ ]:
df_fact_ts_pass_district=pd.merge(left=df_fact_ts_pass,right=df_district,on='dist_code',how='inner')

In [ ]:
fig = plt.figure(figsize =(60, 7))
fig = px.pie(df_fact_ts_pass_district, values='investment in cr', names='district')
fig.show()
print('District wise investment')

In [ ]:
dfs_to_merge = [df_district,df_fact_stamps,df_fact_transport, df_fact_ts_pass]
from functools import reduce

merged_dfs = reduce(lambda left, right: pd.merge(left, right, on='dist_code', how='inner'), dfs_to_merge)





#df_fact_ts_pass_stamps_districts_transport=pd.merge(left=df_district,df_fact_stamps,df_fact_transport,right=df_fact_ts_pass,on='dist_code',how='inner')

In [ ]:
merged_dfs.drop_duplicates(inplace=True)
merged_dfs.info()


In [ ]:
merged_dfs.total_sales=merged_dfs['category_Non-Transport']
merged_dfs.columns

In [ ]:
#10000000
merged_dfs['investment in cr']=merged_dfs['investment in cr']*10000000

In [ ]:
merged_dfs_2021_2022=merged_dfs[(merged_dfs['year']==2021) | (merged_dfs['year']==2022)]

In [ ]:
merged_dfs_2021_2022_relationship=merged_dfs_2021_2022[['district','estamps_challans_rev','investment in cr','total_sales']]

Q10. Relationship between district investments, vehicles
 sales and stamps revenue within the same district between FY 2021
 and 2022

In [ ]:
sns.heatmap(merged_dfs_2021_2022_relationship.drop('district',axis=1).corr(),annot=True)

In [ ]:
districts=merged_dfs_2021_2022_relationship['district'].value_counts().index


for district in districts:
    plt.figure(figsize=(8, 6))
    sns.heatmap(merged_dfs_2021_2022_relationship[merged_dfs_2021_2022_relationship['district']==district].drop('district',axis=1).corr(),annot=True)
    plt.title(f'Correlation Heatmap - {district}')
    plt.show()

There is no district in which the positive correlation value between any two features ('estamps_challans_rev','investment in cr','total_sales'). This indicates there is no relationship between them

Q11. Particular sectors that have shown substantial 
 investment in multiple districts between FY 2021 and 2022. The graph below shows for each particular district distribution of sectors

In [ ]:
for district in districts:
    fig = plt.figure(figsize =(60, 7))
    fig = px.pie(merged_dfs_2021_2022[merged_dfs_2021_2022['district']==district], values='investment in cr', names='sector')
    print(f'Sector wise investment - {district}')
    fig.show()








Q11. Do certain sectors 
 experience higher investments during particular months

In [ ]:
sectors=merged_dfs_2021_2022['sector'].value_counts().index

for sector in sectors:
    fig = plt.figure(figsize =(150, 10))
    merged_dfs_2021_2022[merged_dfs_2021_2022['sector']==sector].groupby(by='month')['fuel_type_others'].sum().to_frame().plot()
    plt.title(f'Sector wise investment - {sector}')
    fig.show()

print('FOR EACH SECTOR IT DISPLAYS TREND OF INVESTMENT ACROSS 2021 AND 2022')

In [ ]:
merged_dfs_2021_2022.head()

In [ ]:
df_fact_ts_pass.head()

In [ ]:
df_fact_transport_district.head()

In [ ]:
df_fact_ts_pass['sector'].value_counts()

In [ ]:
df_tspass_district=pd.merge(left=df_fact_ts_pass,right=df_district,on='dist_code',how='inner')

Top 5 districts to buy commercial properties in 
Telangana

In [ ]:
df_tspass_district[df_tspass_district['sector']=='Real Estate,Industrial Parks and IT Buildings'].groupby(by='district')['investment in cr'].sum().sort_values(ascending=False).to_frame()